# Sentiment Analysis with finetuned distilBERT
Following the [HuggingFace tutorial](https://huggingface.co/docs/transformers/en/tasks/sequence_classification) on text classification using [Goodreads data](https://www.kaggle.com/competitions/goodreads-books-reviews-290312)

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
pip install transformers datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from transformers import pipeline

from datasets import load_dataset

import evaluate

from sklearn.model_selection import train_test_split


## IMDB

In [ ]:
from datasets import load_dataset

imdb = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

## Preprocess & Train

In [ ]:
#tokenize w BERT's tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
#truncate input to the maximum length of BERT's input length

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation = True)


In [ ]:
tokenized_imdb = imdb.map(preprocess_function, batched = True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# now pad the inputs so that each batch is the same length
# do this by batches because otherwise takes too long and uses unnecessary resources
# tbh this function itself is a total black box

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
type(data_collator)

transformers.data.data_collator.DataCollatorWithPadding

In [ ]:
# measure accuracy while training

accuracy = evaluate.load("accuracy")

In [ ]:
#pass predictions and labels to `compute` to calculate the accuracy

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return accuracy.compute(predictions = predictions, references = labels)

In [ ]:
#map ids to labels
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}


In [ ]:
# establish the model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# define training arguments

training_args = TrainingArguments(
    output_dir = "models",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb['train'],
    eval_dataset=tokenized_imdb['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()



wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: michelleamcsweeney (michelleamcsweeney-the-city-university-of-new-york) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.223900,0.230147,0.911280
2,0.142800,0.231459,0.931080


TrainOutput(global_step=3126, training_loss=0.20655757955305865, metrics={'train_runtime': 2045.6985, 'train_samples_per_second': 24.442, 'train_steps_per_second': 1.528, 'total_flos': 6556904415524352.0, 'train_loss': 0.20655757955305865, 'epoch': 2.0})

In [ ]:
#add to my HuggingFace (there must be a million of these)
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/linguanne/models/commit/6ed8e1110dc008f1c590044ad0ea5f11a7c32886', commit_message='End of training', commit_description='', oid='6ed8e1110dc008f1c590044ad0ea5f11a7c32886', pr_url=None, repo_url=RepoUrl('https://huggingface.co/linguanne/models', endpoint='https://huggingface.co', repo_type='model', repo_id='linguanne/models'), pr_revision=None, pr_num=None)

# Test Model

In [ ]:
# my honest review of My Brilliant Friend

text1 = "This is good -- I love listening to the Italian, and the acting is really good. It's hard for a film to be as good as the books, but I think My Brilliant Friend has done it."

In [ ]:
#use my model
classifier = pipeline("sentiment-analysis", model = "linguanne/imdb_tutorial")
classifier(text1)

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9920943379402161}]

In [ ]:
text2 = "too terrifying! I cannot take the psychological horror that is any of the Squid Games."
classifier(text2)

[{'label': 'NEGATIVE', 'score': 0.9715889692306519}]

In [ ]:
text3 = "completely middle of the road"
classifier(text3)

[{'label': 'NEGATIVE', 'score': 0.8130022883415222}]

# GoodReads

In [4]:
from datasets import load_dataset
import datasets

books = (load_dataset("mmcsweeney/goodreads_abbrev", split = "train").train_test_split())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

goodreads_abbrev.csv:   0%|          | 0.00/968M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
books["train"][0]

{'Unnamed: 0': 24876,
 'text': "Short, sweet, and surprisingly not bad. I guess the rating for this one could range from 3-4 stars but in the meantime, I'm giving it three stars just to be fair to my 4-star reads. But yeah, for a novella, it's pretty okay and believe it or not, the story actually flowed beautifully with just the right pacing and development. The blurb actually says it all and it's not like you're expecting to get some epic romance anyway. \n I highly recommend this for those looking for a quick steamy in-betweener :)",
 'label': 1}

In [6]:
#tokenize w BERT's tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
#truncate input to the maximum length of BERT's input length

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation = True)

tokenized_books = books.map(preprocess_function, batched = True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return accuracy.compute(predictions = predictions, references = labels)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/675000 [00:00<?, ? examples/s]

Map:   0%|          | 0/225000 [00:00<?, ? examples/s]

In [7]:


  #map ids to labels
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

# establish the model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)


# define training arguments

training_args = TrainingArguments(
    output_dir="book_review_sentiment",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books['train'],
    eval_dataset=tokenized_books['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: michelleamcsweeney (michelleamcsweeney-the-city-university-of-new-york) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.201700,0.201150,0.925173
2,0.161600,0.209124,0.928360


TrainOutput(global_step=84376, training_loss=0.19853343793630848, metrics={'train_runtime': 15620.0815, 'train_samples_per_second': 86.427, 'train_steps_per_second': 5.402, 'total_flos': 1.762569355097973e+17, 'train_loss': 0.19853343793630848, 'epoch': 2.0})

In [9]:
trainer.push_to_hub()



CommitInfo(commit_url='https://huggingface.co/mmcsweeney/book_review_sentiment/commit/388f8589cbb349fa1010eca9be30c5444a388954', commit_message='End of training', commit_description='', oid='388f8589cbb349fa1010eca9be30c5444a388954', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mmcsweeney/book_review_sentiment', endpoint='https://huggingface.co', repo_type='model', repo_id='mmcsweeney/book_review_sentiment'), pr_revision=None, pr_num=None)

# Test new model

In [1]:
from transformers import pipeline

#use my model
text1 = "this book was fine enough"
classifier = pipeline("sentiment-analysis", model = "mmcsweeney/book_review_sentiment")
classifier(text1)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.6802543997764587}]